# Kadi4Mat Test

This notebook contains some code to upload files to the Kadi4Mat demo instance https://demo-kadi4mat.iam-cms.kit.edu/.

The Tribology example files can be downloaded here: https://zenodo.org/record/6797703.

To run the notebook, create a `.env` file with a variable `ACCESS_TOKEN=...`. You can generate an access token in the Kadi4Mat > User > Settings > Access Tokens.

Not all the files will upload as there is a 25 MB limit on the demo instance.

In [ ]:
# load ACCESS_TOKEN form .env file using python-dotenv
%load_ext dotenv
%dotenv

import os
import json
import requests
from pathlib import Path
from typing import Iterable, Tuple, Dict, Any, Optional

PATH = "/Users/samuel/projects/kadi4mat-test/FAIR Tribological Data of Showcase Pin-on-Disk Experiment"
HOST_URL = "https://demo-kadi4mat.iam-cms.kit.edu"
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

In [ ]:
class KadiSession(requests.Session):
    """Session to interact with Kadi4Mat API."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.base_url = f"{HOST_URL}/api"
        self.headers.update(
            {
                "Content-Type": "application/json; charset=utf-8",
                "Authorization": f"Bearer {ACCESS_TOKEN}",
            }
        )

    def request(self, method, url, *args, **kwargs):
        url = self.base_url + url
        return super().request(method, url, *args, **kwargs)

In [ ]:
def iter_data_files(path: str) -> Iterable[Tuple[Path, Path, Dict[str, Any]]]:
    """Iterate over data files in given directory.

    For each data file yield subdir, json_file and dictionary with the data.
    """

    for subdir in Path(path).iterdir():
        if subdir.is_dir():
            # Find json file in subdirectory
            json_file = subdir.joinpath(f"{subdir.name}.json")
            if json_file.exists():
                # Load json data into dictionary
                with open(json_file, encoding="utf-8") as f:
                    data = json.load(f)

                yield subdir, json_file, data


# for subdir, json_file, data in iter_data_files(PATH):
#     print(subdir, data['identifier'])

In [ ]:
# Helper function to interact with the API


def record_id_by_identifier(session: KadiSession, identifier: str) -> Optional[int]:
    url = f"/records/identifier/{identifier}"
    response = session.get(url)
    return response.json()["id"] if response.status_code == 200 else None


def update_or_create_record(session: KadiSession, data: Dict[str, Any]) -> int:
    identifier = data["identifier"]
    body = {
        "identifier": data["identifier"],
        "title": data["title"],
        "description": data["description"],
        "extras": data["extras"],
        "license": data["license"],
        "tags": data["tags"],
        "type": data["type"],
        "visibility": data["visibility"],
    }
    record_id = record_id_by_identifier(session, identifier)
    if record_id is None:
        # create record
        resp = session.post("/records", json=body)
        record_id = resp.json()["id"]
        print(f"Record {identifier} created. id={record_id}")
    else:
        # update record
        resp = session.patch(f"/records/{record_id}", json=body)
        print(f"Record {identifier} updated.")

    return record_id

In [ ]:
# Update or create records

session = KadiSession()

for subdir, json_file, data in iter_data_files(PATH):
    update_or_create_record(session, data)

In [ ]:
# Create record links


def create_record_links(session: KadiSession, record_id: int, data):
    # Create links
    record_id = record_id_by_identifier(session, data["identifier"])
    for link in data["links"]:
        if link.get("record_to") is not None:
            name = link["name"]
            record_to_id = record_id_by_identifier(
                session, link["record_to"]["identifier"]
            )
            resp = session.post(
                f"/records/{record_id}/records",
                json={"name": name, "record_to": {"id": record_to_id}},
            )
            print("Created link:", resp)


for subdir, json_file, data in iter_data_files(PATH):
    record_id = record_id_by_identifier(session, data["identifier"])
    print(f"@{data['identifier']} ({record_id})")

    # Create record link
    create_record_links(session, record_id, data)

In [ ]:
# Upload files
# Fileupload is a bit more complicated, therefore using kadi-apy to help

from kadi_apy import KadiManager

manager = KadiManager(
    host=HOST_URL,
    token=ACCESS_TOKEN,
)


for subdir, json_file, data in iter_data_files(PATH):
    record = manager.record(identifier=data["identifier"])
    print(f"@{data['identifier']} ({record.id})")

    files_dir = subdir.joinpath("files")
    file_list = record.get_filelist()
    file_names = [f["name"] for f in file_list.json()["items"]]
    # print(f"Existing files: {file_names}")
    if files_dir.exists() and files_dir.is_dir():
        for file in files_dir.iterdir():
            if file.is_file() and file.name not in file_names:
                resp = record.upload_file(str(file))
                print(f"Uploaded file {file.name}. {resp}")